In [2]:
import pandas as pd
import cv2
from ultralytics import YOLO

/tmp/ipykernel_28236/3007640166.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# Load a model
model = YOLO('../../models/yolov8m_goalkeeper.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(['../../data/000135.jpeg'])  # return a list of Results objects

# Process results list
for result in results:
    print(result.boxes.data)  # Boxes object for bbox outputs

In [48]:
import numpy as np
from tqdm import tqdm


file_path = "../../data/raw/arr_embs_id_1.npy"

file_data = np.load(file_path, allow_pickle=True)
df = pd.DataFrame(file_data)
max_len = df.iloc[-1, 0]
#print(df.head())
for i in tqdm(range(430, max_len)):
    embeddet = df.loc[df.iloc[:, 0] == 455].values
    print(embeddet[embeddet[:, 1] == str(2)][0][3])
    break
    for i in embeddet:
        if int(i[1]) == 4:
            print(i)
            print('*****************************************')
    

  0%|          | 0/134570 [00:00<?, ?it/s]

[    0.44266           0           0           0           0           0           0           0           0   0.0088824           0           0           0           0           0           0           0           0     0.20532           0           0           0           0    0.036463           0           0
           0           0           0           0           0           0           0    0.034569           0           0           0           0           0           0           0     0.66408           0           0    0.011008           0           0           0           0           0           0           0
           0           0           0           0           0     0.49394           0           0           0    0.014334     0.24456     0.28281           0      1.1241     0.37739           0           0           0      0.3785           0           0           0           0           0           0           0
     0.24498           0     0.27575           0           0 